In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import  OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql import functions as F, SparkSession
import seaborn as sb
from sklearn.metrics import auc, roc_curve

# Create SparkSession and read CSV file.

In [2]:
spark = SparkSession.builder.appName('Telco Data - Logistic Regression').getOrCreate()

In [3]:
df = spark.read.csv('./telcochurn.csv', header=True, inferSchema=True)

In [4]:
# Drop customer ID column as it is not useful for training a model.
df = df.drop('customerID')

In [5]:
# Convert binary columns of "Yes/No" to "1/0".
df = df.withColumn('Partner_2', F.when(df.Partner == 'Yes', 1).otherwise(0))
df = df.withColumn('Dependents_2', F.when(df.Dependents == 'Yes', 1).otherwise(0))
df = df.withColumn('PhoneService_2', F.when(df.PhoneService == 'Yes', 1).otherwise(0))
df = df.withColumn('PaperlessBilling_2', F.when(df.PaperlessBilling == 'Yes', 1).otherwise(0))
df = df.withColumn('Churn_2', F.when(df.Churn == 'Yes', 1).otherwise(0))

# Convert TotalCharges from strings to doubles.
df = df.withColumn('TotalCharges_2', df.TotalCharges.cast("double"))

In [6]:
# Drop old binary columns.
df = df.drop('Partner', 'Dependents', 'PhoneService', 'Paperlessbilling', 'Churn', 'TotalCharges')

# Use StringIndexer on categorical variable columns.

In [7]:
# Features that are categorical.
categorical_features = ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                        'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

In [8]:
# List of indexers to apply to the data.
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in categorical_features]

In [9]:
# Create and apply the indexers.
pipeline = Pipeline(stages=indexers)
indexed_df = pipeline.fit(df).transform(df)

In [10]:
# Drop original columns.
indexed_df = indexed_df.drop('gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup','DeviceProtection', 
                             'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod')

# Create sparse vectors of each indexed column.

In [11]:
# List of columns that are indexed.
indexed_cols = ['MultipleLines_index', 'InternetService_index', 'OnlineSecurity_index', 'OnlineBackup_index', 
                'DeviceProtection_index', 'TechSupport_index', 'StreamingTV_index', 'StreamingMovies_index', 
                'Contract_index', 'PaymentMethod_index']

In [12]:
# Create encoder.
encoder = OneHotEncoderEstimator(inputCols=indexed_cols,
                                 outputCols=[feat + '_sparse' for feat in indexed_cols])

In [13]:
# Fit and apply encoder to dataframe.
model = encoder.fit(indexed_df)
sparse_indexed_df = model.transform(indexed_df)

In [14]:
# Drop non-sparse indexed columns.
sparse_indexed_df = sparse_indexed_df.drop('MultipleLines_index', 'InternetService_index', 'OnlineSecurity_index',
                                           'OnlineBackup_index', 'DeviceProtection_index', 'TechSupport_index', 
                                           'StreamingTV_index', 'StreamingMovies_index', 'Contract_index', 
                                           'PaymentMethod_index')

# Use VectorAssembler to compress all independent variable columns into a dense vector.

In [15]:
# Create list of IVs.
IV_features = ['SeniorCitizen', 'tenure', 'Paperlessb.illing_2', 'MonthlyCharges', 'TotalCharges_2', 'Partner_2', 
               'Dependents_2', 'PhoneService_2', 'gender_index', 'StreamingTV_index_sparse', 'TechSupport_index_sparse', 
               'PaymentMethod_index_sparse', 'OnlineSecurity_index_sparse', 'InternetService_index_sparse', 
               'StreamingMovies_index_sparse', 'MultipleLines_index_sparse', 'Contract_index_sparse',
               'OnlineBackup_index_sparse', 'DeviceProtection_index_sparse']

In [16]:
# Create the vector assembler and name the vector column "features". The handleInvalid parameter is set to drop rows that 
# contain NA values. In this case, a total of 11 rows are dropped.
vec_assembler = VectorAssembler(inputCols=IV_features, outputCol='features', handleInvalid='skip')

In [17]:
# Create the new dataframe that has the "features" column of dense vectors..
dense_df = vec_assembler.transform(sparse_indexed_df)

IllegalArgumentException: 'Field "Paperlessb.illing_2" does not exist.\nAvailable fields: SeniorCitizen, tenure, MonthlyCharges, Partner_2, Dependents_2, PhoneService_2, PaperlessBilling_2, Churn_2, TotalCharges_2, gender_index, StreamingTV_index_sparse, TechSupport_index_sparse, PaymentMethod_index_sparse, OnlineSecurity_index_sparse, InternetService_index_sparse, StreamingMovies_index_sparse, MultipleLines_index_sparse, Contract_index_sparse, OnlineBackup_index_sparse, DeviceProtection_index_sparse'

In [ ]:
# Create a dataframe consisting only of IV vector and the churn indicator.
processed_df = dense_df.select('features', 'Churn_2')

In [ ]:
# Split the data into three categories: training data, testing data, and validation data.
train, test, valid = processed_df.randomSplit([.6, .2, .2])

# Create, train, and predict with Logistic Regression model.

In [ ]:
# Create Logistic Regression model, fit the model to the training data.
model = LogisticRegression(labelCol='Churn_2')
model = model.fit(train)

In [ ]:
# Get predictions for the testing data and print the accuracy.
test_preds = model.transform(test)

In [ ]:
# Get predictions for the validation data and print the accuracy.
valid_preds = model.transform(valid)

# Calculate accuracy scores.

In [ ]:
def get_accuracy_score(preds_df):
    return preds_df.filter(preds_df.Churn_2 == preds_df.prediction).count() / preds_df.count()

In [ ]:
test_accuracy = get_accuracy_score(test_preds)
valid_accuracy = get_accuracy_score(valid_preds)

In [ ]:
print("Test data prediction accuracy: {0:.3f}".format(test_accuracy))
print("Validation data prediction accuracy: {0:.3f}".format(valid_accuracy))

# Plot confusion matrices.

## Test dataset.

In [ ]:
# Get confusion matrix values.
test_tp = test_preds.filter((test_preds.Churn_2 == 1) & (test_preds.prediction == 1)).count()
test_tn = test_preds.filter((test_preds.Churn_2 == 0) & (test_preds.prediction == 0)).count()
test_fp = test_preds.filter((test_preds.Churn_2 == 0) & (test_preds.prediction == 1)).count()
test_fn = test_preds.filter((test_preds.Churn_2 == 1) & (test_preds.prediction == 0)).count()

In [ ]:
# Create confusion matrix.
test_cm =[[test_tn, test_fp], [test_fn, test_tp]]

## Validation dataset.

In [ ]:
# Get confusion matrix values.
valid_tp = valid_preds.filter((valid_preds.Churn_2 == 1) & (valid_preds.prediction == 1)).count()
valid_tn = valid_preds.filter((valid_preds.Churn_2 == 0) & (valid_preds.prediction == 0)).count()
valid_fp = valid_preds.filter((valid_preds.Churn_2 == 0) & (valid_preds.prediction == 1)).count()
valid_fn = valid_preds.filter((valid_preds.Churn_2 == 1) & (valid_preds.prediction == 0)).count()

In [ ]:
# Create confusion matrix.
valid_cm =[[valid_tn, valid_fp], [valid_fn, valid_tp]]

In [ ]:
## Plot with Seaborn.

In [ ]:
# Set the tickmark names.
classNames = ['Negative','Positive']

# Set the text scale for Seaborn.
sb.set(font_scale=1.6)

In [ ]:
# Set each figure's size.
plt.figure(figsize=(8,8))

#----------
# Test Dataset.
#----------

# Create first subplot.
plt.subplot(1,2,1)

# Set title.
plt.title('Test Dataset')

# Create confusion matrix as a Seaborn heatmap.
sb.heatmap(test_cm, annot=np.array(test_cm), fmt='d', cbar=False, xticklabels=classNames, yticklabels=classNames, square=True)

# Set y tick-label rotation angle.
plt.yticks(rotation=0)

# Set x and y axes names.
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

#----------
# Validation Dataset.
#----------

# Create second subplot.
plt.subplot(1,2,2)

# Set title.
plt.title('Validation Dataset')

# Create confusion matrix as a Seaborn heatmap.
sb.heatmap(valid_cm, annot=np.array(valid_cm), fmt='d', cbar=False, xticklabels=classNames, yticklabels=classNames, square=True)

# Set y tick-label rotation angle.
plt.yticks(rotation=0)

# Set x axis name.
plt.xlabel('Predicted Label')

# Adjust distance from the right edge of each subplot.
plt.subplots_adjust(right=1.7)

plt.show()

# Calculate FPR, TPR, and AUC.

## Test dataset.

In [ ]:
# Create list of 'Churn_2' column values.
test_churn_list = test_preds.select('Churn_2').collect()
test_churn_list = [test_churn_list[i][0] for i in range(len(test_churn_list))]

In [ ]:
# Create list of 'prediction' column values.
test_pred_list = test_preds.select('prediction').collect()
test_pred_list = [test_pred_list[i][0] for i in range(len(test_pred_list))]

In [ ]:
# Get FPR, TPR.
test_fpr, test_tpr, test_thresholds = roc_curve(test_churn_list, test_pred_list)

In [ ]:
# Get AUC.
test_auc = auc(test_fpr, test_tpr)

## Validation dataset.

In [ ]:
# Create list of 'Churn_2' column values.
valid_churn_list = valid_preds.select('Churn_2').collect()
valid_churn_list = [valid_churn_list[i][0] for i in range(len(valid_churn_list))]

In [ ]:
# Create list of 'prediction' column values.
valid_pred_list = valid_preds.select('prediction').collect()
valid_pred_list = [valid_pred_list[i][0] for i in range(len(valid_pred_list))]

In [ ]:
# Get FPR, TPR.
valid_fpr, valid_tpr, valid_thresholds = roc_curve(valid_churn_list, valid_pred_list)

In [ ]:
# Get AUC.
valid_auc = auc(valid_fpr, valid_tpr)

# Plot ROC curves.

In [ ]:
# Clear plots.
plt.clf()

# Set plot and text sizes.
plt.figure(figsize=(10,6))
plt.rcParams.update({'font.size': 13, 'xtick.labelsize' : 18, 'ytick.labelsize' : 18})

#============
# Test ROC curve
#============

# Create first subplot with a white background.
plt.subplot(1,2,1, fc='white')

# ax = plt.axes()
# ax.set_facecolor('white')

# Set title.
plt.title('Test Dataset')

# Plot the data, fpr = x, tpr = y. Bold, with a label for the AUC.
plt.plot(test_fpr, test_tpr, 'b', label = 'AUC = %0.3f' % test_auc)

# Place legend on plot.
plt.legend(loc = 'lower right')

# Plot a dotted red straight line for comparison
plt.plot([0, 1], [0, 1],'r--')

# Set x and y boundaries.
plt.xlim([0, 1])
plt.ylim([0, 1])

# Label axes.
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

#============
# Validation ROC curve
#============

# Create second subplot with a white background.
plt.subplot(1,2,2, fc='white')

# Set title.
plt.title('Validation Dataset')

# Plot the data, fpr = x, tpr = y. Bold, with a label for the AUC.
plt.plot(valid_fpr, valid_tpr, 'b', label = 'AUC = %0.3f' % valid_auc)

# Place legend on plot.
plt.legend(loc = 'lower right')

# Plot a dotted red straight line for comparison
plt.plot([0, 1], [0, 1],'r--')

# Set x and y boundaries
plt.xlim([0, 1])
plt.ylim([0, 1])

# Label x axis.
plt.xlabel('False Positive Rate')

# Adjust distance from the right edge of each subplot.
plt.subplots_adjust(right=1.3)

plt.show()